# Model 8: Predict Mass Class (10) Based on Visual Morphology (clean galaxy image)


In [2]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import ImageFolder 
from torchvision.io import read_image
from torchvision.io import decode_image
from torchvision.models import vgg19
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
from marvin.utils.general.images import get_images_by_list
from marvin import config
from marvin.tools.cube import Cube
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image as image_PIL
import PIL 
from PIL import ImageShow
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, EigenGradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

#set config attributes and turn on global downloads of Marvin data
config.setRelease('DR15')
config.mode = 'local'
config.download = True

import seaborn as sns
sns.set(font_scale=1.6)
sns.set_style('ticks')

%matplotlib qt



[INFO]: No release version set. Setting default to DR15
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


# Importing Data from Schema Table

In [7]:


data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1


#Problem with image associated with manga id at galaxy_list[3548], mangaid- 1-135668
galaxy_list=np.delete(galaxy_list,3548)

galaxy_list=np.unique(galaxy_list)



galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 



galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

galaxies=galaxies.sort_values(by=['plateifu']) #Sorting galaxies by plateifu to match ImageFolder Output 

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

mangaid=galaxies.loc[:,'mangaid']

plateifu=galaxies.loc[:,'plateifu']

if plateifu.all()==np.unique(plateifu).all() and mangaid.all()==np.unique(mangaid.all()):
    print('Is Unique List of Galaxies')
    
else:
    print('Try Again')


z=galaxies.loc[:,'z']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0').reshape(-1,1)

bars=np.linspace(np.min(log_mass),11.25,num=10)
plt.hist(log_mass)
plt.title('Stellar Mass Distribution in SDSS MaNGA')
plt.ylabel('Count')
plt.xlabel('Log Stellar Mass')
plt.vlines(bars,ymin=0,ymax=800)


mass_cat = np.zeros_like(log_mass)
mass_cat[(log_mass> bars[0]) & (log_mass<bars[1])]=int(0)
mass_cat[(log_mass> bars[1]) & (log_mass<bars[2])]=int(1)
mass_cat[(log_mass> bars[2]) & (log_mass<bars[3])]=int(2)
mass_cat[(log_mass> bars[3]) & (log_mass<bars[4])]=int(3)
mass_cat[(log_mass> bars[4]) & (log_mass<bars[5])]=int(4)
mass_cat[(log_mass> bars[5]) & (log_mass<bars[6])]=int(5)
mass_cat[(log_mass> bars[6]) & (log_mass<bars[7])]=int(6)
mass_cat[(log_mass> bars[7]) & (log_mass<bars[8])]=int(7)
mass_cat[(log_mass> bars[8]) & (log_mass<bars[9])]=int(8)
mass_cat[log_mass> bars[9]] = int(9)


print(np.mean(mass_cat))


Is Unique List of Galaxies
4.226525494566732


In [ ]:
print(plateifu.iloc[987])

# Download the Images without Overlays

In [ ]:
for i in range(len(galaxies)):
    im = Image(plateifu.iloc[i])


    # generate a new image
    # inputs are height and width in arcsec, and a arcsec/pixel scale
    im.get_new_cutout(50, 50, scale=0.089)

    # plot the new image cutout
    im.plot()
    plt.savefig('/home/juanp/Documents/Clean Images (Marvin Size Plateifu)/Images/'+str(plateifu.iloc[i]))
    plt.close()


In [ ]:
# Sanity Check Cell
tester=ImageFolder('/home/juanp/Documents/Clean Images (Marvin Size Plateifu)')
i=3549
plt.imshow(tester[i][0])
print(plateifu.iloc[i])

# Importing Images from their Downloaded Locations 


In [3]:
# image_locations=[]
# for i in range (len(galaxy_list)):
#     image_locations.append(Image(galaxy_list[i]).filename)
    
# image_locations=np.array(image_locations,dtype=str)
# np.savetxt('Image Directories',image_locations,fmt='%s')

image_locations=np.loadtxt('Image Directories',dtype=str)



In [3]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    from PIL import Image, ImageOps 
    
    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)
    
    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)
    
    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
  
    # return the resized image
    return new_image



# Putting the Images into DataLoaders

In [6]:
# img_size=(128,128)

# image=ImageFolder('/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/') #Picks up 3590 pictures, directory list has lenght 3637 however 

# torch.manual_seed(15)
# np.random.seed(15)

image_directory='/home/juanp/Documents/Clean Images (Marvin Size Plateifu)'
classes= 10



batch_size=50
image_copies=11

def load_dataset(data_path):
    # Load all the images
    transformation = transforms.Compose([
        # Randomly augment the image data
            # Random horizontal flip
        transforms.RandomHorizontalFlip(0.5),
            # Random vertical flip
        transforms.RandomVerticalFlip(0.3),
        #Rotates the image by some angle
        transforms.RandomRotation(0,360),
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        # transform=transformation
    )
    
    #This loop transforms the images and assigns them the correct label 

    full_dataset_v2 = []   
    for i in range(len(full_dataset)): 
        for j in range(image_copies):
            temp=transformation(resize_image(full_dataset[i][0]))
            full_dataset_v2.append((temp,mass_cat[i])) 
    
    full_dataset=full_dataset_v2

    print(len(full_dataset))


    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    
    # use torch.utils.data.random_split for training/test split
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    
    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
        
    return train_loader, test_loader




#####################################################################################################



# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(image_directory)
batch_size = train_loader.batch_size
print("Data loaders ready to read", image_directory)



39479
Data loaders ready to read /home/juanp/Documents/Clean Images (Marvin Size Plateifu)


# Defining the Model 

In [4]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        
        # In the init function, we define each layer we will use in our model
        
        # Our images are RGB, so we have input channels = 3. 
        # We will apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        
#         self.conv3 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)

#         self.conv4 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # A second convolutional layer takes 12 input channels, and generates 24 outputs
        self.conv5 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # We in the end apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # This means that our feature tensors are now 32 x 32, and we've generated 24 of them
        
        # We need to flatten these in order to feed them to a fully-connected layer
        self.fc = nn.Linear(in_features=24*16*16, out_features=num_classes)

#         self.fc1= nn.Linear(in_features=10, out_features=1)


    def forward(self, x):
        
        
        # In the forward function, pass the data through the layers we defined in the init function
        # print(x)
        # Use a ReLU activation function after layer 1 (convolution 1 and pool)
        x=self.conv1(x)
        
        x=self.pool(x)
       

        x=F.relu(x)

        x = F.relu(self.pool(self.conv2(x)))

#         x = F.relu(self.pool(self.conv3(x)))

#         x = F.relu(self.pool(self.conv4(x)))
      
 
     
        # Use a ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv5(x))) 
        
        
        # Select some features to drop to prevent overfitting (only drop during training)
        x = F.dropout(self.drop(x), training=self.training)
        
        # Flatten
#         print(x.shape)
        
        
        x = x.view(-1, 24*16*16)
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        
        # Return class probabilities via a log_softmax function
#         x=F.relu(x) 
        
#         x= self.fc1(x)
        # print(x)
        return torch.log_softmax(x, dim=1)
    
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

# Create an instance of the model class and allocate it to the device
classes=10
model = Net(num_classes=classes).to('cuda')

print(model)

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=6144, out_features=10, bias=True)
)


# Creating Training Loop/Function

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):

        target=target.long()
        # print(target.shape)
#         target=target.reshape(-1,1)
        
        # Use the CPU or GPU as appropriate
        # Recall that GPU is optimized for the operations we are dealing with
        data, target = data.to(device), target.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
    
        output = model(data)

        
        # Get the loss
        loss = loss_criteria(output, target)
#         if batch_idx==0:
#             print(output,target,output-target,loss)
#         # Keep a running total
        train_loss += loss.item()
        
        # Backpropagate
        loss.backward()
        optimizer.step()

       
        
        # Print metrics so we see some progress
        # print('\tTraining batch {} Loss: {:.6f}'.format(batch_idx + 1, loss.item()))
            
    # return average loss for the epoch
    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

# Create Test Function

In [ ]:
def test(model, device, test_loader):
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            target=target.long()
#             target=target.reshape(-1,1)
            
            batch_count += 1
            data, target = data.to(device), target.to(device)
            
            # Get the predicted classes for this batch
            output = model(data)
            
            # Calculate the loss for this batch
            test_loss += loss_criteria(output, target).item()
            
            # Calculate the accuracy for this batch
            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    # Calculate the average loss and total accuracy for this epoch
    avg_loss = test_loss / batch_count
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # return average loss for the epoch
    return avg_loss

# Training the Model

In [5]:
# Use an "Adam" optimizer to adjust weights
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the loss criteria
loss_criteria = nn.CrossEntropyLoss()

# Track metrics in these arrays
epoch_nums = []
training_loss = []
validation_loss = []




# Train over 10 epochs (We restrict to 10 for time issues)
epochs = 1000
print('Training on', device)
for epoch in range(1, epochs + 1):
        print(epoch)
        train_loss = train(model, device, train_loader, optimizer, epoch)
        test_loss = test(model, device, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)
torch.save(model.state_dict(),'/home/juanp/Documents/SURP-2021/Models/Model 8 (Clean Images and Mass)/MarvinSizePlateifu(11 Copies at 128x128 and 3 layers)(2)')


Training on cuda
1


NameError: name 'train' is not defined

In [ ]:
plt.figure(figsize=(15,15))
plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

In [62]:


# Defining Labels and Predictions
import seaborn as sns
classes=[0,1,2,3,4,5,6,7,8,9]
classes_inverse=[9,8,7,6,5,4,3,2,1,0]
truelabels = []
predictions = []
model.eval()
print("Getting predictions from test set...")
for data, target in test_loader:
    data, target= data.to('cuda'), target.to('cuda')
    for label in target.data.cpu().numpy():
        truelabels.append(label)
    for prediction in model(data).data.cpu().numpy().argmax(1):
        predictions.append(prediction) 

# Plot the confusion matrix
cm = confusion_matrix(truelabels, predictions)
tick_marks = np.arange(len(classes))
cm=np.rot90(cm)

df_cm = pd.DataFrame(cm)
plt.figure(figsize = (7,7))
sns.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='g')
plt.xlabel("Predicted Shape", fontsize = 20)
plt.ylabel("True Shape", fontsize = 20)
plt.yticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5],classes_inverse)


plt.show()



Getting predictions from test set...


# Loading up Saved Model

In [5]:
model.load_state_dict(torch.load('/home/juanp/Documents/SURP-2021/Models/Model 8 (Clean Images and Mass)/MarvinSizePlateifu(11 Copies at 128x128 and 3 layers)(2)'))
model.eval()

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=6144, out_features=10, bias=True)
)

In [ ]:
from torchvision.models import resnet50
model50=resnet50 
print(model50)

In [6]:
image=ImageFolder('/home/juanp/Documents/Clean Images (Marvin Size Plateifu)')
# plt.imshow(image[46][0])
# print(np.shape(image[14][0]))
# PIL.ImageShow.show(resize_image(image[14][0]))

# Check The Class Each Galaxy is Classifed To

In [19]:
i=585
trans=transforms.ToTensor()
input=trans(resize_image(image[i][0]))
input=input.to('cuda')
input=input.unsqueeze(0)
output=model(input)

print(np.where(output.cpu().detach().numpy().flatten()==np.max(output.cpu().detach().numpy().flatten())))
print(plateifu.iloc[i])
print(output)

(array([4]),)
8241-6102
tensor([[-37.3482, -20.7189,  -9.3784,  -3.7425,  -0.5630,  -0.9131,  -5.2115,
         -14.6108, -37.5653, -63.4860]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)


In [ ]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

trans=transforms.ToTensor()

model = model
target_layer = model.conv2

input_tensor =trans(resize_image(image[14][0]))

print(np.max(input_tensor.detach().numpy()))


input_tensor2=np.transpose(input_tensor,(1,2,0))
print(input_tensor2.shape)

input_tensor=input_tensor.unsqueeze(0)
input_tensor=input_tensor.to('cuda')

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layer=target_layer, use_cuda='args.use_cuda')

# If target_category is None, the highest scoring category
# will be used for every image in the batch.
# target_category can also be an integer, or a list of different integers
# for every image in the batch.
target_category = None

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0,:]
visualization = show_cam_on_image(input_tensor2.detach().numpy(), grayscale_cam)
plt.subplot(1,2,1)
plt.imshow(visualization)
plt.subplot(1,2,2)
plt.imshow(resize_image(image[14][0]))

# GradCAM 

In [8]:
image=ImageFolder('/home/juanp/Documents/Clean Images (Marvin Size Plateifu)')
class1=np.where((log_mass> bars[0]) & (log_mass<bars[1]))
class10=np.where(log_mass> bars[9])
print(class1[0][213])
print(len(class1[0]))

all_galaxies=np.arange(0,len(log_mass))
print(all_galaxies[-1])


3500
224
3588


In [16]:
from matplotlib import cm 
viridis = cm.get_cmap('viridis', 10)
import cv2

def galaxy_grad_cam_resize(input_image,file_name,file_name1,model=model):
  
    trans=transforms.ToTensor()

    target_layer = model.conv2

    input_tensor =trans(resize_image(input_image))

    # print(np.max(input_tensor.detach().numpy()))


    input_tensor2=np.transpose(input_tensor,(1,2,0))
    # print(input_tensor2.shape)

    input_tensor=input_tensor.unsqueeze(0)
    input_tensor=input_tensor.to('cuda')

    # Construct the CAM object once, and then re-use it on many images:
    torch.autograd.set_detect_anomaly(True)
    cam = EigenCAM(model=model, target_layer=target_layer, use_cuda='args.use_cuda')
    

    # If target_category is None, the highest scoring category
    # will be used for every image in the batch.
    # target_category can also be an integer, or a list of different integers
    # for every image in the batch.
    target_category = None
    

    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0,:]
    # visualization = show_cam_on_image(input_tensor2.detach().numpy(), grayscale_cam,colormap=cv2.COLORMAP_MAGMA)
    
    plt.figure(figsize=(18,9))
    plt.subplot(1,2,1)
    plt.imshow(grayscale_cam,cmap='viridis')
    plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False) 
    plt.colorbar()
    plt.subplot(1,2,2)
    plt.imshow(resize_image(input_image))
    plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False) 
    plt.savefig('/home/juanp/Documents/Grad-CAM Results/Class 1/Conv2/'+file_name+'.png',transparent=True)
    np.savetxt('/home/juanp/Documents/Grad-CAM Results/Class 1 Focus/Conv2/Map'+file_name1,grayscale_cam)
    plt.close('all')
    return grayscale_cam

j=3099
while j < (len(all_galaxies)):
    i=all_galaxies[j]
    cam=galaxy_grad_cam_resize(image[i][0],'EigenCAM '+str(i),'Map '+str(i))
    j=j+1



In [ ]:
# print(cam.get_cam_weights)
print(cam)
plt.imshow(cam)
plt.colorbar()


In [ ]:
print(plateifu.iloc[4])

In [ ]:


def galaxy_grad_cam(input_image,file_name,model=model):
  
    trans=transforms.ToTensor()

    target_layer = model.conv2

    input_tensor =trans(input_image)

    print(np.max(input_tensor.detach().numpy()))


    input_tensor2=np.transpose(input_tensor,(1,2,0))
    print(input_tensor2.shape)

    input_tensor=input_tensor.unsqueeze(0)
    input_tensor=input_tensor.to('cuda')

    # Construct the CAM object once, and then re-use it on many images:
    torch.autograd.set_detect_anomaly(True)
    cam = EigenCAM(model=model, target_layer=target_layer, use_cuda='args.use_cuda')
    

    # If target_category is None, the highest scoring category
    # will be used for every image in the batch.
    # target_category can also be an integer, or a list of different integers
    # for every image in the batch.
    target_category = None

    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0,:]
    visualization = show_cam_on_image(input_tensor2.detach().numpy(), grayscale_cam)
    plt.figure(figsize=(18,9))
    plt.subplot(1,2,1)
    plt.imshow(visualization)
    plt.subplot(1,2,2)
    plt.imshow(input_image)
    plt.savefig('/home/juanp/Documents/SURP-2021/Plots/Grad-Cam/Mass (Marvin Size)/'+file_name+'.png')
    # plt.close()

i=1225
galaxy_grad_cam(image[i][0],'Image_Eigen'+str(i))

In [ ]:
print(input_tensor.cpu().detach().numpy().shape)
print(grayscale_cam.shape)
print(input_tensor2.detach().numpy().shape)
print(visualization.shape)